# Avaliação e Validação de Modelos

# Por que dividir os dados em conjuntos de dados de treinamento e teste?

## Introdução
Ao desenvolver modelos de aprendizado de máquina, uma das etapas principais é dividir os dados disponíveis em conjuntos de dados de treinamento e teste. Essa prática é crucial por vários motivos, que serão descritos a seguir.

## Motivos para a Divisão de Dados

### 1. **Avaliação do Modelo**

### 2. **Prevenção de Overfitting**

### 3. **Ajuste do Modelo**

### 4. **Validação Cruzada**

### 5. **Comparação Justa**

In [ ]:
import pandas as pd
import requests
from io import StringIO

df = pd.read_csv("https://raw.githubusercontent.com/datasets/cpi-us/main/data/cpiai.csv")

print(df)
# Extraia e defina a coluna 'year' como índice com formato DateTime
df['Date'] = pd.to_datetime(df['Date'])
df.dropna(inplace=True)
df.set_index('Date', inplace=True)
df.sort_index(inplace = True)

df = df["1990-01-01":"2014-01-01"]


# Selecione a coluna 'Index' para os níveis mensais do IPC
cpi_monthly = df['Inflation'].resample('M').mean()

# Exibe as primeiras linhas
print(cpi_monthly.head())

cpi_monthly = cpi_monthly.diff().dropna()

# Dividi os dados em conjuntos de treinamento e teste com base no ponto de divisão
train = cpi_monthly.iloc[:round(len(cpi_monthly)/2)]
test = cpi_monthly.iloc[round(len(cpi_monthly)/2):]

# Exibe as formas do treinamento e dos conjuntos de teste
print("Formato do conjunto de treinamento", train.shape)
print("Forma do conjunto de teste:", test.shape)


In [ ]:
train.sort_index(inplace = True)
train.head()

In [ ]:
train.plot()

In [ ]:
len(test)

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Garante que os dados são apropriados para modelagem
train.dropna(inplace= True) # Remove quaisquer valores faltantes

# Ajusta um modelo autorregressivo (modelo AR)
ar_model = sm.tsa.ARIMA(train, order=(2, 0, 1))
ar_result = ar_model.fit()

print(ar_result.summary())

In [ ]:
train.tail()

In [ ]:
ar_result.forecast(3)

In [ ]:
test.head()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Garante que os dados sejam apropriados para modelagem
train.dropna(inplace=True)  # Remove os valores ausentes dos dados de treinamento

# Ajusta um modelo de média móvel integrada autorregressiva (ARIMA)
arima_model = ARIMA(train, order=(2, 0, 1))
arima_result = arima_model.fit()

# Previsões
# Define manualmente a frequência dos dados da série temporal, se conhecida
freq = 'M' # Substitui 'D' pela frequência apropriada dos seus dados

# Calcula os pontos inicial e final das previsões
start = train.index[-1] + pd.tseries.frequencies.to_offset(freq)
end = start + (len(test) - 1) * pd.tseries.frequencies.to_offset(freq)

predictions = arima_result.predict(start=start, end=end)

# Garante que seus dados de 'test' sejam preparados de forma semelhante aos dados de 'train'
test_prepared = test.dropna()

# Calcula MSE, RMSE e MAPE
mse = mean_squared_error(test_prepared, predictions)
rmse = np.sqrt(mse)
#mape = np.mean(np.abs((test_prepared - predictions) / test_prepared)) * 100

print("MSE:", mse)
print("RMSE:", rmse)
#print("MAPE:", mape)

In [ ]:
print(pd.concat([test_prepared, arima_result.forecast(len(test_prepared))], axis = 1))

# Compreendendo as Métricas de Erro: MSE, RMSE e MAPE

## Erro Quadrático Médio (MSE)
- **Fórmula:** 
  $ \text{MSE} = \frac{1}{n}\sum_{i=1}^{n}(Y_i - \hat{Y}_i)^2 $
  
## Raiz do Erro Quadrático Médio (RMSE)
- **Fórmula:** 
  $ \text{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(Y_i - \hat{Y}_i)^2} $

## Percentual Absoluto Médio Erro (MAPE)
- **Fórmula:** 
  $ \text{MAPE} = \frac{100\%}{n}\sum_{i=1}^{n} \left| \frac{Y_i - \hat{Y}_i}{Y_i} \right| $
- **Interpretação:** O MAPE é fácil de interpretar como uma porcentagem. Valores menores de MAPE indicam melhor ajuste. O MAPE é independente de escala, o que o torna particularmente útil para comparar a precisão entre diferentes conjuntos de dados.